## Train and evaluate model using Scikit-learn and Keras wrapper 

In [1]:
import tensorflow as tf
from aMNWtModel import AMNWtModel


seed = 10
# np.random.seed(seed=seed)  
tf.random.set_seed(seed)

print("---------------------------------------- model ----------------------------------------")
model = AMNWtModel(dataset_file="./Dataset/e_coli_core_UB_100.npz", 
                   objective=['BIOMASS_Ecoli_core_w_GAM'],
                   timestep=4,
                   n_hidden=1,
                   hidden_dim=50,
                   epochs=200, 
                   xfold=5,
                   verbose=True,
                   batch_size=7)
model.printout()

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 200
training regression: True
training droP_out: 0.25
training batch size: 7
training validation iter: 0
training xfold: 5
training early stopping: False


In [2]:
# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
scaler= MinMaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()



from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_validate
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=200, 
                          batch_size=7, 
                          verbose=0)

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

# cross validation
kfold= KFold(n_splits=5,shuffle=True, random_state=seed)
results=cross_validate(estimator, 
                       model.X_train, 
                       model.Y_train, 
                       cv=kfold, 
                       n_jobs=5, 
                       scoring=scoring, 
                       return_train_score=True)
results

2023-09-06 14:30:35.916988: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 14:30:35.918782: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 5. Tune using inter_op_parallelism_threads for best performance.
2023-09-06 14:30:35.931471: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 14:30:35.932858: I tensorflow/core/common_runtime/process_util.cc:

{'fit_time': array([47.76771736, 50.29535532, 45.03954649, 41.84195757, 50.68474698]),
 'score_time': array([0.44260573, 0.43480444, 0.55104494, 0.5660553 , 0.36312604]),
 'test_loss_constraint': array([0.00450187, 0.00430672, 0.00399788, 0.00527716, 0.00453078]),
 'train_loss_constraint': array([0.00446677, 0.00447337, 0.00462127, 0.00428615, 0.00446572]),
 'test_mse': array([0.00386203, 0.00369545, 0.00318019, 0.0045736 , 0.00377568]),
 'train_mse': array([0.00372052, 0.00381115, 0.0038593 , 0.00358447, 0.00372554]),
 'test_R2': array([0.88869158, 0.87925747, 0.89708276, 0.88448292, 0.91322344]),
 'train_R2': array([0.90699752, 0.88847216, 0.91491   , 0.89693221, 0.90542304])}

In [3]:
estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=200, 
                          batch_size=7, 
                          verbose=0)

history = estimator.fit(model.X_train, model.Y_train)
print(history.history["mse"])

from sklearn.metrics import r2_score
y_pred = estimator.predict(model.X_train)
y_true = model.Y_train
y_p = y_pred[:,:y_true.shape[1]]
R2 = r2_score(y_true, y_p, multioutput='variance_weighted')
print(R2)


y_pred = estimator.predict(model.X_test)
y_true = model.Y_test
y_p = y_pred[:,:y_true.shape[1]]
Q2 = r2_score(y_true, y_p, multioutput='variance_weighted')
print(Q2)

Instructions for updating:
Use `tf.linalg.matmul` instead


2023-09-06 14:31:27.054118: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 14:31:27.056414: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-09-06 14:31:27.447160: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


[0.17247505486011505, 0.16161514818668365, 0.15130934119224548, 0.14054113626480103, 0.1288565993309021, 0.1159210130572319, 0.10125409066677094, 0.08521904051303864, 0.06943834573030472, 0.05616830661892891, 0.04603414982557297, 0.038592807948589325, 0.033017586916685104, 0.029195960611104965, 0.026295484974980354, 0.02395368181169033, 0.022071925923228264, 0.02051737532019615, 0.019198892638087273, 0.017907891422510147, 0.01686926744878292, 0.01584378443658352, 0.015039673075079918, 0.014252063818275928, 0.013558387756347656, 0.012966175563633442, 0.012391277588903904, 0.011881964281201363, 0.01143654715269804, 0.011024274863302708, 0.010630127973854542, 0.010285998694598675, 0.00992645975202322, 0.009652146138250828, 0.009348303079605103, 0.009087318554520607, 0.008842726238071918, 0.008627857081592083, 0.008419251069426537, 0.008231722749769688, 0.008057166822254658, 0.007944337092339993, 0.007832787930965424, 0.007602025289088488, 0.007464949507266283, 0.007358833681792021, 0.0072